In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix
from pandas.api.types import is_string_dtype, is_numeric_dtype
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, log_loss, roc_auc_score, roc_curve, auc

# Patch for working directory
path = Path.cwd()

# Get the files from the path provided in the OP
files = Path(path).glob('*.csv')  # .rglob to get subdirectories

In [2]:
#Find all csv files in directory and concatenate them
dfs = list()
names = list()
for f in files:
    data = pd.read_csv(f)
    # .stem is method for pathlib objects to get the filename w/o the extension
    data['file'] = f.stem
    names.append(f.stem)
    dfs.append(data)
    

df_raw = pd.concat(dfs, ignore_index=True)

In [3]:
#drop last column taht shows csv filename source:
del df_raw[df_raw.columns[-1]]

#df_raw

In [4]:
print(df_raw.shape)

df_raw['marker'].value_counts()

(78377, 129)


Attack      55663
Natural     18309
NoEvents     4405
Name: marker, dtype: int64

In [5]:
#----> UNBALANCED DATA

In [6]:
#df_raw.info()

In [7]:
# Look at datatypes of coloumns.
def show_dtypes(df):
    for index in range(len(df.dtypes)):
        print(f'{df.columns[index]} -> {df.dtypes[index]}' )
        
#show_dtypes(df_raw)

In [8]:
# .copy() is very important otherwise its just a pointer
df = df_raw.copy()

In [9]:
def marker_to_numeric(x):
        if x=='NoEvents': return 0
        if x=='Natural':   return 1
        if x=='Attack': return 2

df['marker'] = df['marker'].apply(marker_to_numeric)
#df

In [10]:
np.where(df.values >= np.finfo(np.float64).max)

(array([  738,   739,   753, ..., 78352, 78353, 78353], dtype=int64),
 array([26, 26, 26, ..., 55, 26, 55], dtype=int64))

In [11]:
np.isnan(df.values.any())

False

In [12]:
# Replace infinite updated data with nan
df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Drop rows with NaN
df.dropna(inplace=True)

print(df.shape)

df['marker'].value_counts()

(72073, 129)


2    51445
1    16223
0     4405
Name: marker, dtype: int64

In [13]:
# Due to cleaning up NaNs and infinites we lost about 6000 datapoints, which is not very harmful

In [14]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler

#This only copies a Series Object, no pandas Dataframe!
df_marker = df["marker"].copy()
#Converting to Pandas Dataframe
df_marker = pd.DataFrame(df_marker)

#Delete Marker column because "marker" should not be normalized
del df[df.columns[-1]]

#Initialize SCaler
scaler = StandardScaler()
#Scale data
data_normalized = scaler.fit_transform(df)
#insert normalized data in former pandas frame
df_normalized_nomark = pd.DataFrame(data_normalized, columns=df.columns)

#concat "marker" column with normalized rest dataframe, index of marker frame is overwritten
df_normalized = pd.concat([df_normalized_nomark, df_marker.set_index(df_normalized_nomark.index)], axis=1)

#print(df_normalized_nomark)

#print(isinstance(df, pd.DataFrame))

#print(isinstance(df_marker, pd.DataFrame))

#print(df_normalized)


In [15]:
df_normalized.iloc[:,128]

0        0
1        0
2        0
3        0
4        0
        ..
72068    1
72069    1
72070    1
72071    1
72072    1
Name: marker, Length: 72073, dtype: int64

In [16]:
import torch as T
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable

In [17]:
device = T.device("cpu")

In [18]:
train_df, test_df = train_test_split(df_normalized, test_size=0.3, random_state=42)

train_df.shape, test_df.shape

((50451, 129), (21622, 129))

In [29]:
# Splitting up features and target columns and convert to numpy array

train_X_np = train_df.iloc[:, : -1].to_numpy() # Select all columns except the last one

train_y_np = train_df.iloc[:, -1].to_numpy() # Select the last column

test_X_np = test_df.iloc[:, : -1].to_numpy() 

test_y_np = test_df.iloc[:, -1].to_numpy() 

# Convert to torch tensors

train_X_tensor = T.tensor(train_X_np, dtype=T.float32)

train_y_tensor = T.tensor(train_y_np, dtype=T.long)

test_X_tensor = T.tensor(test_X_np, dtype=T.float32)

test_y_tensor = T.tensor(test_y_np, dtype=T.long)

In [30]:
class MyDataset(Dataset):
    def __init__(self, feature_tensor, target_tensor):
        self.feature_tensor = feature_tensor
        self.target_tensor = target_tensor
        
    def __len__(self):
        return len(self.feature_tensor)
    
    def __getitem__(self, idx):
        features = self.feature_tensor[idx]
        target = self.target_tensor[idx]
        
        return features, target

In [31]:
class ThreeClassesDataset(T.utils.data.Dataset):
    
    def __init__(self, src_file):
        all_xy_np = np.array(src_file.values, dtype=np.float64)
        
        tmp_x_np = all_xy_np[:,0:128]
        
        tmp_y_np = all_xy_np[:,128] 
        
        self.x_data = T.from_numpy(tmp_x_np).to(device)
        
        
        self.y_data = T.from_numpy(tmp_y_np).to(device)
        
    def __len__(self):
        return(self.x_data)
    
    def __getitem__(self,idx):
        preds = self.x_data[idx]
        targets = self.y_data[idx]
        return preds, targets # as a tuple
    

In [32]:
class Net(T.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hid1 = T.nn.Linear(128,10)
        self.hid2 = T.nn.Linear(10,10)
        self.output = T.nn.Linear(10,3)
        
        T.nn.init.xavier_uniform_(self.hid1.weight)
        T.nn.init.zeros_(self.hid1.bias)
        T.nn.init.xavier_uniform_(self.hid2.weight)
        T.nn.init.zeros_(self.hid2.bias)
        T.nn.init.xavier_uniform_(self.output.weight)
        T.nn.init.zeros_(self.output.bias)
        
    def forward(self, x):
        z = T.tanh(self.hid1(x))
        z = T.tanh(self.hid2(z))
        z = T.log_softmax(self.output(z), dim=1) # NLLLoss() 
        return z        

In [41]:
train_ds = MyDataset(train_X_tensor, train_y_tensor)

test_ds = MyDataset(test_X_tensor, test_y_tensor)



bat_size = 1000

train_loader = DataLoader(train_ds, batch_size = bat_size, shuffle=True, drop_last = True)
#shuffle = True is extremely important because training the networks fails throtugh oscillationg biases and weights
#drop_last = True is good practice because the batch sizes should be evenly divided




In [34]:
# create Network
print("Creating 128-(10-10)-3 neural network ")
net = Net().to(device)
net.train()

Creating 128-(10-10)-3 neural network 


Net(
  (hid1): Linear(in_features=128, out_features=10, bias=True)
  (hid2): Linear(in_features=10, out_features=10, bias=True)
  (output): Linear(in_features=10, out_features=3, bias=True)
)

In [35]:
 # 3. train model
max_epochs = 1000
ep_log_interval = 100
lrn_rate = 0.01

loss_func = T.nn.NLLLoss()  # assumes log_softmax()
optimizer = T.optim.SGD(net.parameters(),lr=lrn_rate)

print("bat_size = %3d " % bat_size)
print("loss = " + str(loss_func))
print("optimizer = SGD")
print("max_epochs = %3d " % max_epochs)
print("lrn_rate = %0.3f " % lrn_rate)
print("Starting training ")
for epoch in range(0, max_epochs):
    # T.manual_seed(epoch+1)  # reproducibility
    epoch_loss = 0  # for one full epoch

    for (batch_idx, batch) in enumerate(train_loader):
        X = batch[0]  # inputs
        Y = batch[1]  # correct class/label/politics

        optimizer.zero_grad()
        oupt = net(X)
        loss_val = loss_func(oupt, Y)  # a tensor
        epoch_loss += loss_val.item()  # accumulate
        loss_val.backward()
        optimizer.step()

    if epoch % ep_log_interval == 0:
        print("epoch = %5d  |  loss = %10.4f" % \
        (epoch, epoch_loss))
print("Training done ")

bat_size = 1000 
loss = NLLLoss()
optimizer = SGD
max_epochs = 1000 
lrn_rate = 0.010 
Starting training 
epoch =     0  |  loss =    52.2421
epoch =   100  |  loss =    32.1699
epoch =   200  |  loss =    31.2364
epoch =   300  |  loss =    30.3744
epoch =   400  |  loss =    29.2606
epoch =   500  |  loss =    28.7306
epoch =   600  |  loss =    28.3875
epoch =   700  |  loss =    28.1379
epoch =   800  |  loss =    27.9375
epoch =   900  |  loss =    27.7480
Training done 


In [37]:
def accuracy(model, ds):
    # assumes model.eval()
    # item-by-item version
    n_correct = 0; n_wrong = 0
    for i in range(len(ds)):
        X = ds[i][0].reshape(1,-1)  # make it a batch
        Y = ds[i][1].reshape(1)  # 0 1 or 2, 1D
        with T.no_grad():
            oupt = model(X)  # logits form

        big_idx = T.argmax(oupt)  # 0 or 1 or 2
        if big_idx == Y:
            n_correct += 1
        else:
            n_wrong += 1

    acc = (n_correct * 1.0) / (n_correct + n_wrong)
    return acc

In [42]:
# 4. evaluate model accuracy
print("Computing model accuracy ")
net.eval()
acc_train = accuracy(net, train_ds)  # item-by-item
print("Accuracy on training data = %0.4f" % acc_train)
acc_test = accuracy(net, test_ds) 
print("Accuracy on test data = %0.4f" % acc_test)

Computing model accuracy 
Accuracy on training data = 0.7371
Accuracy on test data = 0.7370
